In [1]:
from pathlib import Path
from typing import List
import json

src_url='https://www.industrydocuments.ucsf.edu/food/results/#q=*%3A*&h=%7B%22hideDuplicates%22%3Afalse%2C%22hideFolders%22%3Atrue%7D&subsite=food&cache=true&count=164846'
num_pages = 25 # each page consists of 20 samples

res_path = "../dataset/ucsf-idl"

(Path(res_path) / "images").mkdir(exist_ok=True, parents=True)

In [2]:
from scraper import scrape_url

samples = scrape_url(src_url, num_pages=num_pages)


In [ ]:
from tqdm import tqdm
import requests
from pdf2image import convert_from_bytes

for sample in tqdm(samples):
    id = sample["id"]
    download_url = sample["download_url"]

    img_folder = Path("images") / str(id)
    img_path = Path(res_path) / img_folder
    img_path.mkdir(exist_ok=True, parents=True)

    sample["image_folder"] = str(img_folder)

    r = requests.get(download_url, allow_redirects=True)
    bytes = r.content

    #convert to img and save
    images = convert_from_bytes(bytes)
    for i, img in enumerate(images):
        img.save(img_path / f"page_{i}.jpg", "JPEG")


100%|██████████| 500/500 [16:33<00:00,  1.99s/it]


In [11]:
# Create labels 

import json


allowed_types = ["email", "letter", "agenda", "comments", "report", "conference proceedings", "article", "memo", 'photograph']
allowed_num_types = 1


samples = json.load(open( Path(res_path) / "samples.json"))

labels = []
for sample in samples:
    label = {}
    label["id"] = sample["id"]

    if "img_path" in sample:
        label["image_folder"] = sample["image_folder"]
    else:
        label["image_folder"] = str(Path("images") / str(sample["id"]))

    label["pages"] = sample["pages"]    
    
    types = sample["type"].replace(",", "").replace(";", "").split(" ")
    types = [types] if len(types)== 0 else types
    
    # Filter with type count
    filtered_types = [t for t in types if t in allowed_types]
    if len(filtered_types) == allowed_num_types:
        label["type"] = filtered_types[0] 
    else:
        continue

    labels.append(label)

# overwrite labels  
json.dump(labels, open(Path(res_path) / "labels.json", "w+"))